In [2]:
!pip install git+https://github.com/quantopian/pyfolio --quiet

You should consider upgrading via the '/home/user/notebook/jupyterenv/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import pyfolio as pf
# import matplotlib.pyplot as plt
# import datetime
import FinanceDataReader as fdr
# import warnings
# import matplotlib.cbook
# warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# holding list 가져오기

In [3]:
def getHoldingsList(indexName):
    stocks = list(fdr.StockListing(indexName)['Symbol'] )# 나스닥
    return stocks[:30] # 갯수 바꿀 수 있음!!

In [4]:
stocks = getHoldingsList('KOSPI')
stocks

['095570',
 '006840',
 '152100',
 '295820',
 '253150',
 '253160',
 '395750',
 '395760',
 '278420',
 '292750',
 '309210',
 '333940',
 '333950',
 '333960',
 '333970',
 '333980',
 '269530',
 '251590',
 '161510',
 '251600',
 '289670',
 '298340',
 '189400',
 '278620',
 '269540',
 '287180',
 '213630',
 '332610',
 '332620',
 '195970']

# 리스트를 데이터프레임으로 바꾸기

In [5]:
def getCloseDatafromList(stocks, start_date):
    df = pd.DataFrame()
    for s in stocks:
        df[s] = fdr.DataReader(s, start_date)['Close'] 
    return df

In [6]:
prices = getCloseDatafromList(stocks, '2021-01-01')
prices

,095570,006840,152100,295820,253150,253160,395750,395760,278420,292750,...,289670,298340,189400,278620,269540,287180,213630,332610,332620,195970
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,4580,25250,40117,10102,44800,4755,NaN,NaN,8369,18526,...,57245,53470,15900,105780,15250,19120,12580,91935,99000,11275
2021-01-05,4935,25050,40741,10289,46125,4610,NaN,NaN,8465,18776,...,57375,53530,15790,105815,15045,19085,12470,92460,99870,11340
2021-01-06,4710,24900,40284,10254,45165,4705,NaN,NaN,8446,18594,...,57200,53555,15900,105820,15060,18985,12550,92225,98780,11355
2021-01-07,4695,25200,41217,10476,47590,4485,NaN,NaN,8649,18987,...,57165,53495,15845,105825,15305,19035,13025,92460,96900,11515
2021-01-08,4540,25350,43095,10623,51600,4065,NaN,NaN,8785,19816,...,57150,53495,16205,105815,15525,19630,13080,92720,96630,11470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-30,5860,27800,42915,12100,49715,3940,9940.0,9920.0,9870,19505,...,57280,53480,17625,106025,17960,23470,15265,97700,103200,12260
2021-08-02,6000,27200,43160,12070,50260,3905,9990.0,9995.0,9825,19550,...,57190,53480,17640,106025,18125,24045,15140,98050,103505,12295
2021-08-03,5990,26850,43550,12070,51150,3840,10025.0,10030.0,9820,19765,...,57155,53475,17575,106020,18040,24020,15135,97630,103205,12305


# 현재 듀얼 모멘텀으로 어떤 주식을 사야하는지 list

In [7]:
def bool_converter(bool_var):
    """Returns Integer Value from Boolean Value
    Parameters
    ----------
    bool_var : boolean
        Boolean variables representing trade signals
    Returns
    -------
    result : int
        Integer variables representing trade signals
    """
    if bool_var == True:
        result = 1
    elif bool_var == False:
        result = 0
    return result

In [8]:
def DualMomentum(prices, lookback_period, n_selection):
    # lookback_period: 몇개월 모멘텀 이용할건지
    # n_selection: Top 몇개의 종목을 선택할건지(상대모멘텀)
    
    # absolute momentum
    returns = prices.pct_change(periods=lookback_period).fillna(0)
    long_signal = (returns > 0).applymap(bool_converter)
    abs_signal = long_signal[-1:]

    # relative momentum
    returns = prices.pct_change(periods=lookback_period).fillna(0)
    rank = returns.rank(axis=1, ascending=False)
    long_signal = (rank <= n_selection).applymap(bool_converter)
    rel_signal = long_signal[-1:]

    signal = (abs_signal == rel_signal).applymap(bool_converter) * abs_signal
    dual_momentum_list = list(signal[signal==1].dropna(axis=1).columns)
    return dual_momentum_list

In [9]:
dualmomentumlist = DualMomentum(prices, lookback_period = 20, n_selection = len(stocks)//2) 
dualmomentumlist

['095570',
 '152100',
 '295820',
 '253150',
 '333940',
 '333950',
 '333970',
 '333980',
 '251590',
 '161510',
 '289670',
 '269540',
 '287180',
 '332620',
 '195970']